In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torchvision import models, transforms, datasets
import cv2
import os
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

In [3]:
img_size = (224,224)
batch_size = 1
num_classes = 10

val_transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.CenterCrop(img_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [5]:
val_dataset = datasets.CIFAR10(root='data', train=False, download=True, transform=val_transform)
test_dataset, val_dataset = train_test_split(val_dataset, test_size=0.2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model = models.vgg16(pretrained=True)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

model.load_state_dict(torch.load('myModel.pth'))


<All keys matched successfully>

In [23]:
actual_labels = []
predicted_labels = []
probs = []
with torch.no_grad():
    for inputs, labels in tqdm(test_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        predicted_probs = F.softmax(outputs, dim=1).detach().to('cpu').numpy()
        predicted_labels.append(predicted.item())
        actual_labels.append(labels.item())
        probs.append(predicted_probs)
probs = np.array(probs)
probs=probs.reshape(-1, 10)       

100%|███████████████████████████████████████| 8000/8000 [01:50<00:00, 72.25it/s]


In [26]:
accuracy = accuracy_score(actual_labels, predicted_labels)
auc = roc_auc_score(actual_labels, probs, average='weighted', multi_class='ovr')
precision = precision_score(actual_labels, predicted_labels, average='weighted')
recall = recall_score(actual_labels, predicted_labels, average='weighted')
confusionmatrix = confusion_matrix(actual_labels, predicted_labels)

print("The accuracy is: ", accuracy)
print("The ROC-AUC score is: ", auc)
print("The precision is: ", precision)
print("The recall is: ", recall)
print("The confusion matrix is: ")
print(confusionmatrix)

The accuracy is:  0.879
The ROC-AUC score is:  0.9918372446636456
The precision is:  0.8802457409345767
The recall is:  0.879
The confusion matrix is: 
[[685   5  23  11  11   2   5   9  40   3]
 [  6 767   2   2   0   0   3   0   6   8]
 [ 16   0 661  18  31  16  47  12   1   0]
 [  5   2  34 634  28  51  33  13   2   0]
 [  2   0  24  10 709   8  18  25   0   0]
 [  1   0  33  69  20 652  11  28   0   0]
 [  3   0  14  12  14   2 763   0   0   0]
 [  4   0   9  13  21  11   4 733   0   0]
 [ 28   3  11   2   4   0   4   0 742   4]
 [ 16  59   3   8   0   1   2   3  19 686]]
